In [1]:
from utils import Activities, Users, InfoParser

import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import logging
logging.basicConfig(level=logging.DEBUG)

from config_vars import month_map, month_order
import math
import datetime

def is_nan(x):
    return isinstance(x, float) and math.isnan(x)

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows    = None

In [3]:
data_folder = "../data/"
raws_folder = "../data/mysql_extractions/"

In [4]:
inii = InfoParser("../report/data_info.ini")

# Data Loading and Cleaning

### Loading Feedbacks

In [5]:
feedbacks = pd.read_csv(f"{raws_folder}activities_feedbacks_info.csv")

In [6]:
def get_year_school(date):
    if is_nan(date):
        return np.nan
    
    datee = datetime.datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
    
    if datee.month < 8:
        return datee.year-1
    
    return datee.year

In [7]:
feedbacks['request_year'] = pd.DatetimeIndex(feedbacks['request_date']).year
feedbacks['response_year'] = pd.DatetimeIndex(feedbacks['response_date']).year

feedbacks['request_year_school'] = feedbacks['request_date'].apply(get_year_school)
feedbacks['response_year_school'] = feedbacks['response_date'].apply(get_year_school)

### Cleaning Feedbacks

remove feedbacks that doesn't have a user

In [8]:
u_f = Users()
u_f.load_correct_map(f'{data_folder}duplicate_users_map.pckl')

feedbacks['sender'] = u_f.correct_users(feedbacks['sender'])

In [9]:
users = pd.read_csv(f"{data_folder}users.csv",sep='\t')
users['has_user'] = True

feedbacks = feedbacks.merge(users[['us_user','has_user']],left_on="sender",right_on="us_user",how="left")

In [10]:
before = len(feedbacks)
feedbacks.dropna(subset=["has_user"],inplace=True)
after = len(feedbacks)

inii.info('feedbacks_no_user',before-after)

print(f"{before-after} feedbacks removed, {after} feedbacks remained")

67 feedbacks removed, 5131 feedbacks remained


In [11]:
feedbacks.drop(['has_user'],axis=1,inplace=True)

# Export extracted data

In [12]:
feedbacks.to_csv(f"{data_folder}activities_feedbacks_info.csv",index=False)